<a href="https://colab.research.google.com/github/asongulol/big_data/blob/master/ETL_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.4.6'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Packa

In [3]:
# Download Postgresql driver
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-09-01 01:15:39--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-09-01 01:15:40 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

# Extract Transform and Load (ETL)

## Extract

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)

# Create a dataframe
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), inferSchema=True, sep="\t", timestampFormat="yyyy-mm-dd", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-01-14 00:10:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

## Transform

In [7]:
# Count number of rows
df.count()

3105520

In [8]:
# Drop null values
df = df.dropna()

print(df.count())

3105360


In [9]:
# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [10]:
# Changing datatypes
# Source Link: https://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark

from pyspark.sql.types import TimestampType, IntegerType

df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", df["total_votes"].cast(IntegerType()))\

# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [11]:
# Create review dataframe to match review_id_table table
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RQ58W7SMO911M|   12076615|0385730586|     122662979|2005-01-14 00:10:00|
|  RF6IUKMGL8SF|   12703090|0811828964|      56191234|2005-01-14 00:10:00|
|R1DOSHH6AI622S|   12257412|1844161560|     253182049|2005-01-14 00:10:00|
| RATOTLA3OF70O|   50732546|0373836635|     348672532|2005-01-14 00:10:00|
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717|2005-01-14 00:10:00|
|R2F53LI9KK9MOY|   31048862|0316769487|     862964341|2005-01-14 00:10:00|
|R1KJ6MB7MRSQFF|   53000124|0805076069|     145341889|2005-01-14 00:10:00|
|R2XIM9LT335WHE|   29732693|1581603681|     640542054|2005-01-14 00:10:00|
|R1VE0FQQ0QTQJN|   48541186|0300108834|     915891133|2005-01-14 00:10:00|
|R1VKEE2NWSWDRU|   15280864|0446531081|     880645124|2005-01-14 00:10:00|
|R2UP6XSVYJBJ2H|   242092

In [12]:
# Print schema
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [13]:
# Create product dataframe to match products table(only unique values)
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0029148510|Acts of War: Beha...|
|1844494896|Waiting For Kate ...|
|0451216954|Dark Lover (Black...|
|006056718X|Often Wrong, Neve...|
|0965852768|101 Spooktacular ...|
|141960435X|Flip Flop Spanish...|
|0471282502|Experimental Orga...|
|0756613647|Universe: The Def...|
|B0006YGCE8|Seismic risk asse...|
|207030602X|L Etranger (Folio...|
|1569711224|Aliens: Hive  (2n...|
|0756613744|Go Figure!: A Tot...|
|0891073825|The Cross: God's ...|
|1595140840|            Teach Me|
|1844491412|Led Zeppelin: The...|
|0060737980|Surrounded by Idi...|
|0486265250|Ancient Egyptian ...|
|0763627798|           Shoe Baby|
|097559950X|Natural Cures "Th...|
|0595362478| Accursed Inhabitant|
+----------+--------------------+
only showing top 20 rows



In [14]:
# Create customer dataframe to match customers table
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   50122160|         21922|
|   50732546|          9964|
|   52615377|          2664|
|   45041039|          2215|
|   50776149|          1797|
|   50913245|          1786|
|   20595117|          1641|
|   45273033|          1622|
|   39569598|          1416|
|   49693975|          1384|
|   38491967|          1336|
|   36642996|          1300|
|   50881246|          1263|
|   52254603|          1159|
|   51987970|          1039|
|   49750558|          1036|
|   49115690|          1023|
|   50608834|          1006|
|   50068216|           997|
|   51147790|           996|
+-----------+--------------+
only showing top 20 rows



In [21]:
# Create vine dataframe to match vine_table table

# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# Filter for only columns from Amazon's Vine program
#vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|
|R2XIM9LT335WHE|          4|            3|         10|   N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|
|R2UP6XSVYJBJ2H|          5|            0|          0|   N|
|R21SYDQ70ILUC0|          5|            1|          1|   N|
| R7M06Z88PD7SX|          4|           14|         14|   N|
| RRS38KZ4WB5O2|          5|            

In [20]:
# Filter for only columns from Amazon's Vine program
Y_vine_table_df = vine_table_df.filter(col("vine")  == "Y")
Y_vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2NKNL4WXWIWBG|          4|            3|          3|   Y|
| RAIG9ZQDMBF5P|          5|            3|          6|   Y|
+--------------+-----------+-------------+-----------+----+



## Load

In [22]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.clw759vfh14c.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "f9dLM910&%S%", 
          "driver":"org.postgresql.Driver"}

In [23]:
# Write DataFrame to review_id_df table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [24]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [25]:
# Write DataFrame to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [26]:

# Write DataFrame to vine_table table in RDS

vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

# Natural Language Processing (NLP)